# Getting data

In order to obtain our data regarding crypto currencies we will use Messari's API so that it provides us with up to date information on the matter.

### REST API

We will request Messari.io for top assets and their market price back one year. For that we will need to first obtain the assets and then ask for their historic data.

In [1]:
import requests, json

url = "https://data.messari.io/api/v1/assets?fields=id,slug,symbol,metrics/market_data/price_usd"
response = requests.get(url)
assets = json.loads(response.text)['data']

Now that we know our assets we can request their history.

In [2]:
from datetime import date
import pandas as pd

# Time frame
today = date.today()
yearago = today.replace(year = today.year -1)

# Iterate for each asset
for a in assets:
    asset = a['slug']
    url = f'https://data.messari.io/api/v1/assets/{asset}/metrics/price/time-series?start={yearago}&end={today}&interval=1d'
    response = requests.get(url)
    info = json.loads(response.text)
    # Make sure some data is returned
    if "data" in info:
        columns = info["data"]["parameters"]["columns"]
        data = info["data"]["values"]
        df = pd.DataFrame(data, columns=columns)
        df["slug"] = asset
        df.to_csv(f"../data/{today}-{asset}.csv")

In [3]:
df.head()

,timestamp,open,high,low,close,volume,slug
0,1609977600000,36896.930000,40360.874663,33934.066431,39310.169566,1.483733e+08,wrapped-bitcoin
1,1610064000000,39245.590000,42138.620000,36481.869186,40613.409135,1.755506e+08,wrapped-bitcoin
2,1610150400000,40683.410000,41392.940000,38783.730000,40374.328448,7.739323e+07,wrapped-bitcoin
3,1610236800000,40419.568319,171525.105292,35125.000000,38191.359776,1.379977e+08,wrapped-bitcoin
4,1610323200000,38260.470000,38315.580000,30100.703692,35422.147375,3.634866e+08,wrapped-bitcoin


The information contains the timestamp; open, highest, lowest and closing prices in USD and the volume of operations.

# Binance

Binance also has its own python client that can be used for this same data retrieval.

In [4]:
from binance.client import Client
client = Client()
info = client.get_all_tickers()
info[:5]

[{'symbol': 'ETHBTC', 'price': '0.07589100'},
 {'symbol': 'LTCBTC', 'price': '0.00317500'},
 {'symbol': 'BNBBTC', 'price': '0.01079100'},
 {'symbol': 'NEOBTC', 'price': '0.00057100'},
 {'symbol': 'QTUMETH', 'price': '0.00249100'}]

Same operation may be done, asking for each asset its historical information. For this example we will just filter some of the assets to retrieve information from.

In [5]:
# Asset list we are interested in
asset_list = ['BNBUSDT','BTCUSDT','ETHUSDT','SOLUSDT','ADAUSDT','XRPUSDT','DOTUSDT','DOGEUSDT']

# Time frame
today = date.today()
yearago = today.replace(year = today.year -1).strftime("%Y.%m.%d")
today = today.strftime("%Y.%m.%d")
timeframe="1d"

# Iterate for each asset
for tick in info:
    asset = tick["symbol"]
    # We will filter the assets to work with
    if asset in asset_list:
        data = client.get_historical_klines(asset, timeframe, yearago, today)
        df = pd.DataFrame(data, columns=["Open time","Open","High","Low","Close","Volume", "Closing time","Quote asset vol", "Num traders", "Taker buy base asset vol", "Taker buy quote asset vol","To be ignored"])
        df["asset"] = asset
        df.to_csv(f"../data/{today}-{asset}.csv")